<a href="https://www.kaggle.com/code/bennyav/learn-ai-today-01?scriptVersionId=181713204" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# 01 - Getting started with PyTorch

https://towardsdatascience.com/learn-ai-today-01-getting-started-with-pytorch-2e3ba25a518

In [60]:
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML
import plotly.graph_objects as go
from torch import optim
import torch.nn.functional as F



## Linear Regression

In [ ]:
import torch
from torch import nn

# Original code uses Module from fastai
class LinearRegression(nn.Module):
    def __init__(self, number_of_inputs, number_of_outputs):
        super().__init__()
        self.linear = nn.Linear(number_of_inputs, number_of_outputs)

    def forward(self, x):
        return self.linear(x)




In [ ]:
import numpy as np

def fit(inputs, targets, model, criterion, optimizer, num_epochs):
    """
    Train the `model` with the train dataset `(inputs, targets)`
    'criterion' in the function used to calculate the loss
    `optimizer` changes the model weights based on the loss
    """
    loss_history = []
    out_history = []
    
    first_iteration = True
    
    for epoch in range(num_epochs):
        # forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        
        # backward pass
        optimizer.zero_grad()
        # what exactly is done on this step
        loss.backward()
        # How the optimizer is aware of the loss
        optimizer.step()
        
        loss_history.append(loss.item())
        #print("out:", outputs.detach().numpy())
        if first_iteration:
            out_history = outputs.detach().numpy()
            first_iteration=False
        else:
            out_history = np.concatenate((out_history, outputs.detach().numpy()), axis=1)
        #print('Epoch {}/{}, Loss {:.6f}'.format(epoch, num_epochs, loss.item()))
    return (out_history, loss_history)
        

In [ ]:
model = LinearRegression(1,1)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)

In [ ]:
# prepare the training data

x_train = torch.linspace(0,1,10000)
true_y = 2 * x_train + 1  # True relationship
# Observed y is 2X+1 + some noise
y_train = 2*x_train + 1 + torch.randn(x_train.size())*x_train*0.1
x_train_ready = x_train.unsqueeze(-1)
y_train_ready =  y_train.unsqueeze(-1)

## Visualization functions

In [ ]:
# Just a static plot of noisy data (Observations) grey scatter, real function - blue line, and predicted function - red line
def matplot_prediction_ground_noise(x, prediction, ground_truth, observations):
    plt.plot(x, ground_truth, color='blue')
    plt.plot(x, prediction, color='red')
    plt.scatter(x, observations, color='gray')
    plt.grid()
    plt.legend(['Prediction', 'Observation'])
    

In [ ]:
def plotly_prediction_ground_noise(x, *, prediction=None, ground_truth=None, observations=None):
    """
    Plots observation, ground truth, and predictions over a graph using plotly
    """
    # Create the figure
    fig = go.Figure()
    
    Legend = []
    if observations is not None:
        # Add noisy data points
        fig.add_trace(go.Scatter(x=x, y=observations, mode='markers', name='Noisy data (model inputs)', marker=dict(color='gray')))

    if ground_truth is not None:
        # Add true data line
        fig.add_trace(go.Scatter(x=x, y=ground_truth, mode='lines', name='True data', line=dict(color='black')))
        
    if prediction is not None:
        # Add predicted line
        fig.add_trace(go.Scatter(x=x, y=prediction, mode='lines', name='Prediction', line=dict(color='rgba(255, 0, 0, 0.5)')))

    # Update layout
    fig.update_layout(
        title='True Data vs Noisy Data',
        xaxis_title='X',
        yaxis_title='Y = 2x + 1',
        legend=dict(x=0.01, y=0.99, bgcolor='rgba(255,200,255,0.5)')
    )

    # Show the plot
    fig.show()

In [ ]:
def plotly_loss_over_time(loss_history):
    # what happens to the loss over epochs
    loss_fig = go.Figure()
    loss_fig.add_trace(go.Scatter(x=[i for i in range(len(loss))],y=loss_history, mode='lines', name='Loss Per Epoch', line=dict(color='red') ))
    loss_fig.update_layout(
        title='Loss per Epoch',
        xaxis_title='Epoch',
        yaxis_title='Loss',
        legend=dict(x=0.01, y=0.99, bgcolor='rgba(255,200,255,0.5)'),
        showlegend=True
    )
    loss_fig.show()

In [ ]:
def plot_animated_prediction(x, *, ground_truth, observations, prediction_per_epoch):
    """
    Plot animation of prediction per epoch together with the ground truth and observations (Noise)
    """
    fig, ax = plt.subplots(figsize=(6,3), dpi=120)
    #ax.set_xlim((-0.1, 1.1))
    #ax.set_ylim((-5, 20))
    ax.plot(x, ground_truth, lw=2, color='black', label='True model')
    ax.scatter(x, observations, s=10, alpha=0.5, color='gray', label='Observed data') 
    ax.set_ylabel(r'$y = 3x^2 + 2x + 1 + noise$') # Notice you can use latex in the label string
    line, = ax.plot([], [], lw=2, label='Predicted model')
    ax.legend()

    # animation function. This is called sequentially
    def animate(i):
        line.set_data(x, prediction_per_epoch[...,i])
        return (line,)

    # call the animator. blit=True means only re-draw the parts that have changed.
    anim = animation.FuncAnimation(fig, animate, frames=out.shape[1], interval=30, blit=True)
    return anim

In [ ]:
# Define the data for visualization
x = x_train.numpy()
true_y = 2 * x + 1  # True relationship

noisy_y = y_train.numpy()

In [ ]:
%%time
# Alternatively use the above plotly function
plotly_prediction_ground_noise(x, ground_truth=true_y, observations=noisy_y)

## Let's get our hands dirty

In [ ]:
# Train

# what is requires_grad?
(out, loss) = fit(inputs=x_train_ready.requires_grad_(True), targets=y_train_ready, model=model, criterion=criterion, optimizer=optimizer, num_epochs=200)

In [ ]:
plotly_loss_over_time(loss)

In [ ]:
# Let's plot the most uptodate over the trainning data
# interestingly, before calling this, the model didn't generate it correctly
#model.eval()
#outputs = model(x_train_ready)

# Create the figure
animated_fig = go.Figure()

# Add noisy data points
noisy_trace = go.Scatter(x=x, y=noisy_y, mode='markers', name='Noisy data (model inputs)', marker=dict(color='gray'))
animated_fig.add_trace(noisy_trace)

# Add true data line
true_data_trace = go.Scatter(x=x, y=true_y, mode='lines', name='True data', line=dict(color='black'))
animated_fig.add_trace(true_data_trace)

# Add the prediction line
prediction_trace = go.Scatter(x=[None], y=[None], mode='lines', name='Epoch Prediction', line=dict(color='blue'))
animated_fig.add_trace(prediction_trace)

frames = []
for i in range(len(out[0])):
    frames.append(go.Frame(data=[
        noisy_trace,
        go.Scatter(x=x_train.numpy(), y=out[..., i], mode='lines', name='Epoch Prediction', line=dict(color='blue'))],
                 name=f'Epoch {i}'))

animated_fig.update(frames=frames)
animated_fig.update_layout(
    title='True Data vs Noisy Data',
    xaxis_title='X',
    yaxis_title='Y = 2x + 1',
    legend=dict(x=0.01, y=0.99, bgcolor='rgba(255,255,255,0.5)'),
    updatemenus=[{
        'type': 'buttons',
        'showactive': False,
        'buttons': [{
            'label': 'Play',
            'method': 'animate',
            'args': [None, {'frame': {'duration': 50, 'redraw': False}, 'fromcurrent': True}]
            #'args': [None]
        }]
    }])
animated_fig.show()
#animated_fig.write_html(file='animated_plot.html', auto_open=True)

In [ ]:
%%capture 
anim = plot_animated_prediction(x, ground_truth=true_y, observations=noisy_y, prediction_per_epoch=out)


In [ ]:
HTML(anim.to_html5_video())

# Let's do ploynomial regression

In [ ]:
# prepare the data

poly_x_true = torch.linspace(-2, 2, 1000)
poly_y_true = 3*poly_x_true**2 + 2*poly_x_true + 1
poly_y_train = poly_y_true + torch.randn(poly_y_true.size())
poly_y_train.unsqueeze_(-1)

# The way to predict a polynom with linear regression is by having two free variables: x, and X^2
poly_x_train = torch.cat((poly_x_true.unsqueeze(-1)**2, poly_x_true.unsqueeze(-1)), dim=1)




In [ ]:
plotly_prediction_ground_noise(poly_x_true, ground_truth=poly_y_true, observations=poly_y_train.squeeze(1))

In [ ]:
# Train
poly_model = LinearRegression(2, 1)
criterion = nn.MSELoss()
optimizer = optim.Adam(poly_model.parameters(), lr=0.1)

%time (out, loss) = fit(inputs=poly_x_train, targets=poly_y_train, model=poly_model, criterion=criterion, optimizer=optimizer, num_epochs=250)

In [ ]:
plotly_loss_over_time(loss)

In [ ]:
poly_model.eval()
ye = poly_model(poly_x_train)
ye=ye.detach().squeeze(1).numpy()

In [ ]:
plotly_prediction_ground_noise(poly_x_true, ground_truth=poly_y_true, observations=poly_y_train.squeeze(1), prediction=ye)

In [ ]:
%%capture 
anim = plot_animated_prediction(poly_x_true, ground_truth=poly_y_true, observations=poly_y_train, prediction_per_epoch=out)

In [ ]:
HTML(anim.to_html5_video())

In [ ]:
print(list(poly_model.parameters()))


## Neural Network

In [ ]:
class GeneralFit(nn.Module):
    def __init__(self, input_size, output_size, hidden_size=100):
        super().__init__()
        self.linear_in = nn.Linear(input_size, hidden_size)
        self.hidden = nn.Linear(hidden_size, hidden_size)
        self.linear_out = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        x = torch.relu(self.linear_in(x))
        x = torch.relu(self.hidden(x))
        y = self.linear_out(x)
        
        return y
        
        

## Cubic Polynom

In [ ]:
# prepare the data

cubic_x = torch.linspace(-2, 2, 1000).unsqueeze(-1)
cubic_y_true = 6*cubic_x**3 + 3*cubic_x**2 + 2*cubic_x + 1
cubic_y_train = cubic_y_true + torch.randn(cubic_y_true.size())

print(cubic_x.size(), cubic_y_train.size())

In [ ]:
print(cubic_x[..., 0].size())

In [ ]:
# Lets plot true and noisy Y
plotly_prediction_ground_noise(cubic_x.squeeze(1), ground_truth=cubic_y_true.squeeze(1), observations=cubic_y_train.squeeze(1))

In [ ]:
# Train the model
cubic_model_nn = GeneralFit(1,1)
criterion = nn.MSELoss()
optimizer = optim.Adam(cubic_model_nn.parameters(), lr=0.05)

%time (cubic_nn_out, cubic_nn_loss) = fit(inputs=cubic_x, targets=cubic_y_train, model=cubic_model_nn, criterion=criterion, optimizer=optimizer, num_epochs=500)

In [ ]:
plotly_loss_over_time(cubic_nn_loss)

In [ ]:
cubic_model_nn.eval()
%time cubic_nn_y_prediction = cubic_model_nn(cubic_x).detach()


In [ ]:
# Plot prediction alongside observations and ground truth
plotly_prediction_ground_noise(cubic_x.squeeze(1), 
                               ground_truth=cubic_y_true.squeeze(1), 
                               observations=cubic_y_train.squeeze(1),
                               prediction=cubic_nn_y_prediction.squeeze(1))

In [ ]:
%%capture 
anim = plot_animated_prediction(cubic_x.squeeze(1), 
                                ground_truth=cubic_y_true.squeeze(1), 
                                observations=cubic_y_train.squeeze(1), 
                                prediction_per_epoch=cubic_nn_out)


In [ ]:
HTML(anim.to_html5_video())

## Cubic Polynom with a linear regression model
We just used the `GeneralFit` model to train for a cubic polynom.
Can we use the linear regression model we used earlier for the same purpose using the same method we used to train it for quadratic polynom

In [ ]:
# The way to predict a polynom with linear regression is by having three free variables: x, and X^2
cubic_x_linear = torch.cat((cubic_x**3, cubic_x**2, cubic_x), dim=1)

# Train the model
cubic_model_linear = LinearRegression(3,1)
criterion = nn.MSELoss()
optimizer = optim.Adam(cubic_model_linear.parameters(), lr=0.01)

%time (cubic_linear_out, cubic_linear_loss) = fit(inputs=cubic_x_linear, targets=cubic_y_train, model=cubic_model_linear, criterion=criterion, optimizer=optimizer, num_epochs=250)

In [ ]:
plotly_loss_over_time(cubic_linear_loss)